# N-Gram Models

In [116]:
from random import randint
from math import log, exp, prod
from pprint import pprint

from sklearn.model_selection import train_test_split
from nltk import ngrams
from indicnlp.tokenize.indic_tokenize import trivial_tokenize
from indicnlp.tokenize.sentence_tokenize import sentence_split

## Normalizing the text

## Maximum Likelihood Estimation
- The maximum likelihood of a sequence $$w_{i-n+1},\dots ,w_{i-2},w_{i-1},w_i$$ is the likelihood of $w_i$ coming after the sequence $$w_{i-n+1},\dots ,w_{i-2},w_{i-1}$$
- Thus,
$$
P(w_i|w_{i-n+1},\dots ,w_{i-2},w_{i-1}) = \frac{
    C(w_{i-n+1},\dots ,w_{i-2},w_{i-1},w_i)
    }{
        \sum_w C(w_{i-n+1},\dots ,w_{i-2},w_{i-1},w)
    }
$$
- Now, the number of n-grams starting with $w_{i-n},\dots ,w_{i-2},w_{i-1}$ is the same as the number of times the sequence appears, thus $$  \sum_w C(w_{i-n},\dots ,w_{i-2},w_{i-1},w) = C(w_{i-n},\dots ,w_{i-2},w_{i-1}) $$
- Thus, the MLE becomes
$$
P(w_i|w_{i-n+1},\dots ,w_{i-2},w_{i-1}) = \frac{
    C(w_{i-n+1},\dots ,w_{i-2},w_{i-1},w_i)
    }{
        C(w_{i-n+1},\dots ,w_{i-2},w_{i-1})
    }
$$

In [117]:
class NGramModel:
    def __init__(self, text: str, n: int, keep_punctuation: bool=False, verbose: bool=False):
        self.text = text
        self.n = n
        self.verbose = verbose
        self.keep_punctuation = keep_punctuation
        sentences, words = self._tokenize(text)
        self.vocabulary = list(set(words))
        self.n_grams = self._make_n_grams(sentences, n)
        self.n1_grams = self._make_n_grams(sentences, n-1)
        # if self.verbose:
            # pprint(self.n_grams)

    def _tokenize(self, text: str):
        sentences = [
            trivial_tokenize(sentence)
                for sentence in sentence_split(text, lang='hi')
        ]
        
        punctutation=['.', ',', '!', '?', '।']
        words = [
            word
                for sentence in sentences
                    for word in sentence
        ]

        sentences = [(
            ['<s>']*(self.n-1) + sentence + ['</s>']*(self.n-1)
        ) for sentence in sentences]

        if not self.keep_punctuation:
            sentences = [
                [word for word in sentence
                    if word not in punctutation]
                for sentence in sentences
            ]
            words = [
                word for word in words
                    if word not in punctutation
            ]
        return sentences, words

    def _make_n_grams(self, sentences: list[list[str]], n: int) -> list[tuple[str, ...]]:
        n_grams = [
            ng for sentence in sentences
                for ng in ngrams(sentence, n)
        ]
        return n_grams

    def getMLE(self, word: str, prev_words: tuple[str]) -> float:
        if len(prev_words) >= self.n:
            raise ValueError('prev_words should have length n-1')
        if len(prev_words) < self.n-1:
            prev_words = ('<s>',)*(self.n-1-len(prev_words)) + prev_words
        sequence = tuple(prev_words+(word,))
        count_sequence = self.n_grams.count(sequence)
        count_prev = self.n1_grams.count(prev_words)
        if count_prev == 0:
            mle = 0
        else:
            mle = count_sequence/count_prev
        if self.verbose:
            pprint({
                'seq': sequence,
                'cs': count_sequence,
                'prev': prev_words,
                'cp': count_prev,
                'mle': mle
            })
        return mle

    def getSentenceProb(self, sentence: str) -> float:
        tokenized_sentence, _ = self._tokenize(sentence)
        s = tokenized_sentence[0]
        prob_pairs = [(
            sent,
            tuple(s[i-self.n+1:i])
        ) for i, sent in enumerate(s)][self.n-1:]
        prob = prod([self.getMLE(*pair) for pair in prob_pairs])
        # prob = exp(sum([log(self.getMLE(*pair)) for pair in prob_pairs]))
        return prob
    
    def _generateWord(self, last_n_words: tuple[str], top_n):
        word_probs = [(word, self.getMLE(word, last_n_words)) for word in self.vocabulary]
        best_words = sorted(word_probs, key=lambda x: x[1], reverse=True)[:top_n]
        # print(best_words)
        rand_ind = randint(0, top_n - 1)
        next_word = best_words[rand_ind][0]
        return next_word

    def generateNextWord(self, words: str, top_n: int):
        last_n_words = words[-self.n: ]
        return self._generateWord(tuple(last_n_words), top_n)

    def generateText(self, seed_words: str, length: int, top_n: int=3):
        last_n_words = seed_words[-self.n: ]
        text = " ".join(seed_words) + " "
        for i in range(length):
            new_word = self._generateWord(tuple(last_n_words), top_n)
            last_n_words = last_n_words[1:] + [new_word]
            text += new_word + " "
        return text


## MLE Prediction

In [118]:
text = ''
with open('datasets/iit_bombay/dev_test/dev.hi', 'r') as file:
    text = file.read()

In [119]:
text = text.split('\n')
_, test_text = train_test_split(text, train_size=0.7)
text = "\n".join(text)
test_text = "\n".join(test_text)

In [120]:
print(len(text.split('\n')))
text

521


'महानगर पालिका अंतर्गत दत्तात्रय नगर माध्यमिक स्कूल के विद्यार्थियों ने काल्पनिक किला \'दत्तगढ़\' बनाकर अपनी कल्पनाशक्ति का परिचय दिया।\nप्रधानाध्यापक संध्या मेडपल्लीवार के प्रोत्साहित करने पर शिक्षकों व विद्यार्थियों ने मिट्टïी से किले का निर्माण किया।\nमनपा शिक्षक संघ के अध्यक्ष राजेश गवरे ने स्कूल को भेंट देकर सराहना की।\nकिले का परीक्षण रमेश सातपुते ने किया।\nकिला निर्माण में निखिल कावले, दर्शन गेड़ेकर, साहिल मेश्राम इन विद्यार्थियों ने सहभाग लिया।\nजिला कलाध्यापक संघ के अध्यक्ष नरेंद्र बारई, कोषाध्यक्ष शेखर वानस्कर, सदस्य अजय गुंडमवार, गजानन मेहर ने विद्यार्थियों का मार्गदर्शन किया।\nनगरसेवक रीता मुले ने सदिच्छा भेंट दी।\nरोहतक. नौकरियों में भ्रष्टाचार, फर्जीवाड़े व लूट- खसोट के खिलाफ अखिल भारतीय जनवादी महिला समिति और डीवाईएफआई ने संयुक्त रूप से प्रदेशव्यापी अभियान शुरू किया है।\nइस राज्यव्यापी हस्ताक्षर अभियान के माध्यम से प्रदेश भर में 10 लाख हस्ताक्षर करवाकर राज्यपाल को सौंपे जाएंगे।\nशुक्रवार को नए बस स्टैंड पर हस्ताक्षर अभियान की शुरुआत की गई।\nइस मौके पर एसएफआई के राज्य सचिव

In [121]:
print(len(test_text.split('\n')))
test_text

157


"उन्होंने इस मामले की जाच करते हुए 13वें वित आयोग की राशि पर तत्काल रोक लगाने की मांग की है।\nगुरुवार को ट्रकों की सघन जाच हो रही थी।\nप्रमुख ने कहा कि सरकारी कार्य में पारदर्शिता रखकर विकास कार्य को आगे बढ़ाने की जरूरत है।\nजिला कलाध्यापक संघ के अध्यक्ष नरेंद्र बारई, कोषाध्यक्ष शेखर वानस्कर, सदस्य अजय गुंडमवार, गजानन मेहर ने विद्यार्थियों का मार्गदर्शन किया।\nउन्होंने कहा कि पाकिस्तान की भारत के प्रति हीन भावना ही है, जो बार-बार उसे गलत मार्ग पर चलने को मजबूर कर रही है।\nइसी मौके पर हम ये कुछ तस्वीरें गुजरात से चुनकर लाए हैं।\nहमें हीरे की बिक्री पिछले साल की तुलना में इस साल 25 फीसदी अधिक रहने की उम्मीद है।\nरात लगभग दो बजे बमरौली कटारा चौकी प्रभारी रजिस्टर पाल सिंह यादव घायल सिपाहियों को जीप से डौकी लेकर आ रहे थे।\nउन्होंने कहा कि ऐसे फैसलों के लिए राजनीतिक इच्छाशक्ति और कड़े निर्णय की जरूरत पड़ती है।\nहिटलर ने आत्मकथा 'मीन कॉफ' में लिखा है कि झूठ का आकार भरोसे का मुख्य कारण होता है।\nजागरण संवाद केंद्र, राजौरीः ड्यूटी में लापरवाही बरतने वाले सरकारी कर्मचारियों पर शिकंजा कसने के लिए

In [122]:
bigram_model = NGramModel(text, 2)

In [123]:
bigram_model.getMLE('समय', ('उस',))

0.6666666666666666

## Sentence Prediction

In [124]:
bigram_model.getSentenceProb('शुक्रवार को छोटी दिवाली यानी की धनतेरस है।')

4.6126843477351237e-10

In [125]:
bigram_model.getSentenceProb('शुक्रवार को छोटी दिवाली यानी की धनतेरस है')

4.6126843477351237e-10

In [126]:
bigram_model.getSentenceProb('स्कूल को छोटी दिवाली यानी की धनतेरस है।')

1.4192874916108074e-11

## Text Generation

In [127]:
bigram_model.generateNextWord(['को'], 3)

'इस'

In [128]:
bigram_model.generateText(['को'], 20)

'को भी हैं तो बिजली निगम ने अपने मंत्रिमंडल का आयात शुल्क मूल्य बढ़ाकर 738 सभ्यता और बरगी विधानसभा को लेकर '

In [129]:
bigram_model.generateText(['पर'], 20)

'पर शांति सभ्यता और अनारक्षित वेस्ट सभ्यता वेस्ट वेस्ट वेस्ट दिलबाग वेस्ट दिलबाग वेस्ट सभ्यता सभ्यता और पेंशन सभ्यता वेस्ट दिलबाग '

In [130]:
bigram_model.vocabulary

['सभ्यता',
 'वेस्ट',
 'संवैधानिक',
 'लाभ',
 'रिवीजन',
 'रोगी',
 'विश्वविद्यालय',
 'अकाउंटेंट्स',
 'जब',
 'ओवर',
 'जरिए',
 'शिकंजा',
 'उनकी',
 'मृदुला',
 'डाली',
 'सवाल',
 'मैदान',
 'बीच',
 'कह',
 'पाई',
 'दास',
 'मर्जी',
 'ग्राम',
 'खरीदते',
 'हजारों',
 '1.5',
 'प्रत्याशियों',
 'बागवानी',
 'रहे',
 'लगे',
 'सुनाई',
 'महीने',
 'प्रकार',
 '31',
 'सामने',
 'अभियान',
 'चोरी',
 'सेल',
 'गुम',
 'महाप्रबंधक',
 'केन्ट',
 'हरीश',
 'रेवन्यू',
 'काफी',
 'चचेरी',
 'अभेद',
 'अजय',
 'लगातार',
 'सप्ताह',
 'चली',
 'किताब',
 'नागरिक',
 'जिस',
 'डीटीओ',
 'सहकारिता',
 'महापुरुष',
 'बार',
 'प्रेरक',
 'आज',
 'कानून',
 'पद',
 'टैक्सी',
 'चीख',
 'कराए',
 'मुनाफे',
 'दुर्गा',
 'बताने',
 'खिलाफ',
 'रिजर्व',
 'मुहल्ला',
 'सोनकर',
 'फीस',
 'बाड़वाला',
 'ऊपर',
 'ऑफ',
 'जेल',
 'उखाड़कर',
 'अधिक',
 'राजव्यवस्थाएं',
 'वनस्पति',
 'विकसित',
 'स्कूल',
 'जोड़ा',
 'टांगों',
 'एवं',
 'भरना',
 'मिला',
 'पीएम',
 'चलाकर',
 'देनी',
 'नौकरशाही',
 'संयुक्त',
 'खराब',
 'साहित्य',
 'समझनी',
 'स्वागत',
 "'",
 'ड्यूटी',
 'बढ़ती',
 '

## Trigram

In [131]:
trigram_model = NGramModel(text, 3)
trigram_model.generateText(['के', 'बाद'], 20, top_n=10)

'के बाद होने लाभ सभ्यता सभ्यता वेस्ट रिवीजन विश्वविद्यालय जब रोगी वेस्ट ओवर रोगी ओवर जब ओवर अकाउंटेंट्स सभ्यता सभ्यता अकाउंटेंट्स लाभ '

## Testing the Model

In [132]:
test_sents, _ = bigram_model._tokenize(test_text)

In [135]:
testing_set = [sent[1:3] for sent in test_sents]
preds = [bigram_model.generateNextWord(word[:-1], top_n=1) for word in testing_set]
actuals = [word[-1] for word in testing_set]
acc = [1 if pred == actual else 0 for pred, actual in zip(preds, actuals)]
accuracy = sum(acc)/len(acc)
accuracy

0.4936708860759494

In [136]:
testing_set = [sent[1:4] for sent in test_sents]
print(testing_set)
preds = [trigram_model.generateNextWord(word[:-1], top_n=1) for word in testing_set]
actuals = [word[-1] for word in testing_set]
acc = [1 if pred == actual else 0 for pred, actual in zip(preds, actuals)]
accuracy = sum(acc)/len(acc)
accuracy

[['उन्होंने', 'इस', 'मामले'], ['गुरुवार', 'को', 'ट्रकों'], ['प्रमुख', 'ने', 'कहा'], ['जिला', 'कलाध्यापक', 'संघ'], ['उन्होंने', 'कहा', 'कि'], ['इसी', 'मौके', 'पर'], ['हमें', 'हीरे', 'की'], ['रात', 'लगभग', 'दो'], ['उन्होंने', 'कहा', 'कि'], ['हिटलर', 'ने', 'आत्मकथा'], ['जागरण', 'संवाद', 'केंद्र'], ['अपने', 'आप', 'से'], ['इसमें', 'राजनीति', 'ही'], ['पंजाब', 'से', 'करीब'], ['उसकी', 'दोनों', 'टांगों'], ['अमेरिकी', 'अर्थव्यवस्था', 'को'], ['अंजली', 'का', 'कहना'], ['तोरा', 'चौकी', 'के'], ['स्कूल', 'प्रबंधक', 'के'], ['प्रभु', 'श्रीराम', 'के'], ['परिवार', 'का', 'कहना'], ['कांग्रेस', 'अपनी', 'विरासत'], ['सीडीपीओ', 'कार्यालय', 'का'], ['पार्टी', 'की', 'जारी'], ['भारतीय', 'जनता', 'पार्टी'], ['शोभन', 'सरकार', 'ने'], ['मतदाता', 'सरकारों', 'से'], ['कन्नूर', 'देश', 'का'], ['कांग्रेसी', 'मित्रों', 'से'], ['अभिभावकों', 'का', 'कहना'], ['हवन', 'व', 'श्रीमद्कथा'], ['पंजाब', 'सरकार', 'ने'], ['इन', 'सभी', 'कर्मचारियों'], ['जांच', 'न्यूनतम', 'ध्वनि'], ['खरीदारों', 'के', 'रुख'], ['हरीश', '10वीं', 'की'], ['इस', 'प

0.8227848101265823

In [137]:
trigram_model.generateText(['उन्होंने', 'इस'], length=100)

'उन्होंने इस मामले सभ्यता वेस्ट वेस्ट संवैधानिक संवैधानिक सभ्यता सभ्यता सभ्यता संवैधानिक वेस्ट वेस्ट संवैधानिक सभ्यता सभ्यता वेस्ट वेस्ट सभ्यता वेस्ट वेस्ट सभ्यता वेस्ट वेस्ट सभ्यता सभ्यता सभ्यता संवैधानिक संवैधानिक संवैधानिक संवैधानिक संवैधानिक वेस्ट वेस्ट संवैधानिक वेस्ट सभ्यता संवैधानिक वेस्ट सभ्यता संवैधानिक वेस्ट सभ्यता वेस्ट वेस्ट वेस्ट वेस्ट संवैधानिक सभ्यता सभ्यता सभ्यता वेस्ट सभ्यता संवैधानिक सभ्यता सभ्यता सभ्यता संवैधानिक संवैधानिक वेस्ट वेस्ट सभ्यता संवैधानिक संवैधानिक वेस्ट वेस्ट वेस्ट सभ्यता सभ्यता वेस्ट वेस्ट वेस्ट वेस्ट सभ्यता सभ्यता सभ्यता वेस्ट सभ्यता सभ्यता वेस्ट सभ्यता संवैधानिक सभ्यता वेस्ट सभ्यता सभ्यता सभ्यता सभ्यता वेस्ट सभ्यता संवैधानिक वेस्ट संवैधानिक सभ्यता सभ्यता सभ्यता सभ्यता सभ्यता सभ्यता सभ्यता सभ्यता '